In [3]:
import os
import json
import pandas as pd
import traceback

In [14]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192",temperature=0.5)

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain 

## implementing callback in groq

In [7]:
from typing import Any, Dict, List

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate

class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        if serialized is not None:
            print(f"Chain {serialized.get('name')} started")
        else:
            print("Chain started with no serialized data")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")

callbacks = [LoggingHandler()]
prompt = ChatPromptTemplate.from_template("What is 1 + {number}?")

chain = prompt | model

# Ensure callbacks are configured correctly
chain_with_callbacks = chain.with_config(callbacks=callbacks)

# Invoke the chain with input
chain_with_callbacks.invoke({"number": "2"})

Chain started with no serialized data
Chain ChatPromptTemplate started
Chain ended, outputs: messages=[HumanMessage(content='What is 1 + 2?')]
Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='The answer to 1 + 2 is 3.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='The answer to 1 + 2 is 3.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 18, 'total_tokens': 31, 'completion_time': 0.010833333, 'prompt_time': 0.002565972, 'queue_time': 0.017806698, 'total_time': 0.013399305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b9842ce-ec89-4dce-9cfb-6981d6f19830-0', usage_metadata={'input_tokens': 18, 'output_tokens': 13, 'total_tokens': 31}))]] llm_output={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 18, 'total_tokens': 31, 'completion_time': 0.010833333, 'prompt_time': 0.002565972, 'queue_time': 0.

AIMessage(content='The answer to 1 + 2 is 3.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 18, 'total_tokens': 31, 'completion_time': 0.010833333, 'prompt_time': 0.002565972, 'queue_time': 0.017806698, 'total_time': 0.013399305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b9842ce-ec89-4dce-9cfb-6981d6f19830-0', usage_metadata={'input_tokens': 18, 'output_tokens': 13, 'total_tokens': 31})

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
from langchain_core.prompts import PromptTemplate

In [10]:
template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
    )

In [12]:
quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Chotu chapri\AppData\Local\Temp\ipykernel_11532\2075755995.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [13]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=template2)

In [15]:
review_chain=LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path = r"D:\gen ai\mcq_generator\data.txt"

In [18]:
file_path

'D:\\gen ai\\mcq_generator\\data.txt'

In [19]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]

Born on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat the Devil Fruit that turned hi

In [21]:
#Serialize the Python dicitonary to JSON-formatted string
RESPONSE_JSON=json.dumps(RESPONSE_JSON)

In [41]:
TEXT = TEXT.replace("\n", " ")
NUM_QUESTIONS = 4
SUBJECT = "Monkey D Luffy"
TONE = "casual"

In [42]:
class LoggingHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.responses = []
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

        #Assuming response contains token information
        if hasattr(response, 'usage'):
            self.total_tokens += response.usage.get('total_tokens', 0)
            self.responses.append(response)

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        if serialized is not None:
            print(f"Chain {serialized.get('name')} started")
        else:
            print("Chain started with no serialized data")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")
        print(f"Total tokens used: {self.total_tokens}")

In [43]:
logging_handler = LoggingHandler()

In [44]:
response = generate_evaluate_chain.invoke({"text": TEXT, "number": NUM_QUESTIONS, "subject": SUBJECT, "tone": TONE, "response_json": RESPONSE_JSON}, callbacks=[logging_handler])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]  Born on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Monkey D Luffy students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Here is the quiz in JSON format:

{"1": {"mcq": "What is Monkey D. Luffy's nickname?", "options": {"a": "Straw Hat Luffy", "b": "Pirate King", "c": "Rubber Man", "d": "Fire Fist Luffy"}, "correct": "a"}, 
"2": {"mcq": "Who is Monkey D. Luffy's greatest inspiration?", "options": {"a": "Portgas D. Ace", "b": "Monkey D. Garp", "c": "Shanks", "d": "Monkey D. Dragon"}, "correct": "c"}, 
"3": {"mcq": "What is Monkey D. Luffy's goal in life?", "options": {"a": "To become the Pirate King"

In [45]:
print(response)

{'text': 'Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]  Born on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat the Devil Fruit that

In [46]:
print(f"Total tokens used: {logging_handler.total_tokens}")

Total tokens used: 0


In [47]:
quiz = response.get("quiz")

In [48]:
print(quiz)

Here is the quiz in JSON format:

{"1": {"mcq": "What is Monkey D. Luffy's nickname?", "options": {"a": "Straw Hat Luffy", "b": "Pirate King", "c": "Rubber Man", "d": "Fire Fist Luffy"}, "correct": "a"}, 
"2": {"mcq": "Who is Monkey D. Luffy's greatest inspiration?", "options": {"a": "Portgas D. Ace", "b": "Monkey D. Garp", "c": "Shanks", "d": "Monkey D. Dragon"}, "correct": "c"}, 
"3": {"mcq": "What is Monkey D. Luffy's goal in life?", "options": {"a": "To become the Pirate King", "b": "To find the legendary treasure of Gol D. Roger", "c": "To save the world from the Marines", "d": "To become the strongest warrior"}, "correct": "a"}, 
"4": {"mcq": "Who enabled Luffy to eat the Devil Fruit that turned his body into rubber?", "options": {"a": "Portgas D. Ace", "b": "Sabo", "c": "Shanks", "d": "Curly Dadan"}, "correct": "c"}}

Let me know if you need any changes!


In [52]:
#lines = quiz.splitlines()
#quiz_without_first_line = "\n".join(lines[1:])

In [53]:
quiz_new = pd.DataFrame(quiz.split("\n"), columns=["quiz"])

In [54]:
quiz_new.to_csv("quiz.csv", index=False)

In [4]:
from datetime import datetime
datetime.now().strftime('%d-%m-%Y %H:%M:%S')

'28-12-2024 15:12:49'